In [1]:
import pandas as pd
df_1 = pd.read_csv('../data/4.processed_data/Query4_results_test_processed.csv')
df_2 = pd.read_csv('../data/4.processed_data/Query1503_results_conv_processed.csv')
df_3 = pd.read_csv('../data/4.processed_data/Query1603_results_test_processed.csv')
df_train = pd.concat([df_1,df_2,df_3])
df_test = pd.read_csv('../data/4.processed_data/Query1803_results_processed.csv')

In [2]:
# Calculate the average of rel_diff_to_min_price_FlightID for each Journey_ID and Detected_Country
mean_savings_Journey_ID_train = df_train.groupby(['Journey_ID','Detected_Country'])['rel_diff_to_min_price_FlightID'].mean().reset_index(name='mean_savings_for_Journey_ID_country')
# Merge the average_savings DataFrame back into df_reduced
# Calculate the average of rel_diff_to_min_price_FlightID for each Journey_ID and Detected_Country
median_savings_Journey_route_train = df_train.groupby(['Journey_route','Detected_Country'])['rel_diff_to_min_price_FlightID'].median().reset_index(name='median_savings_for_Journey_route_country')
# Merge the average_savings DataFrame back into df_reduced



# Calculate the average of rel_diff_to_min_price_FlightID for each Journey_ID and Detected_Country
mean_savings_Journey_ID_test = df_test.groupby(['Journey_ID','Detected_Country'])['rel_diff_to_min_price_FlightID'].mean().reset_index(name='mean_savings_for_Journey_ID_country')
# Merge the average_savings DataFrame back into df_reduced

# Calculate the average of rel_diff_to_min_price_FlightID for each Journey_ID and Detected_Country
median_savings_Journey_route_test = df_test.groupby(['Journey_route','Detected_Country'])['rel_diff_to_min_price_FlightID'].median().reset_index(name='median_savings_for_Journey_route_country')
# Merge the average_savings DataFrame back into df_reduced

In [3]:
# Step 1: Create a copy of df_reduced to work with
df_enriched = df_train.copy()

# Assuming mean_savings_Journey_ID and median_savings_Journey_route are as previously defined

# Step 2: Merge the mean_savings_for_Journey_ID_country DataFrame into df_enriched
df_enriched = df_enriched.merge(mean_savings_Journey_ID_train, on=['Journey_ID', 'Detected_Country'], how='left')

# Step 3: Merge the median_savings_Journey_route into df_enriched
# Note: Rename 'median_savings_for_Journey_ID_country' to 'median_savings_for_Journey_route' to avoid confusion
df_enriched = df_enriched.merge(median_savings_Journey_route_train, on=['Journey_route', 'Detected_Country'], how='left')

# Step 4: Calculate the simple mean of the two savings values
df_enriched['mean_savings'] = df_enriched[['mean_savings_for_Journey_ID_country', 'median_savings_for_Journey_route_country']].mean(axis=1)

In [4]:

# Step 2: Merge the mean_savings_for_Journey_ID_country DataFrame into df_enriched
df_test = df_test.merge(mean_savings_Journey_ID_test, on=['Journey_ID', 'Detected_Country'], how='left')

# Step 3: Merge the median_savings_Journey_route into df_enriched
# Note: Rename 'median_savings_for_Journey_ID_country' to 'median_savings_for_Journey_route' to avoid confusion
df_test = df_test.merge(median_savings_Journey_route_test, on=['Journey_route', 'Detected_Country'], how='left')

# Step 4: Calculate the simple mean of the two savings values
df_test['mean_savings'] = df_test[['mean_savings_for_Journey_ID_country', 'median_savings_for_Journey_route_country']].mean(axis=1)

In [5]:
# Assuming `df_train` is your training dataset and `df_test` is your test dataset

# Calculate mean savings for departure and destination airports on the training data
mean_savings_by_departure = df_enriched.groupby('departure_airport_code')['mean_savings'].mean()
mean_savings_by_destination = df_enriched.groupby('destination_airport_code')['mean_savings'].mean()

# Map these encodings onto both training and test sets
df_enriched['departure_airport_encoded'] = df_enriched['departure_airport_code'].map(mean_savings_by_departure)
df_enriched['destination_airport_encoded'] = df_enriched['destination_airport_code'].map(mean_savings_by_destination)

df_test['departure_airport_encoded'] = df_test['departure_airport_code'].map(mean_savings_by_departure)
df_test['destination_airport_encoded'] = df_test['destination_airport_code'].map(mean_savings_by_destination)

global_mean_savings = df_enriched['mean_savings'].mean()

df_test['departure_airport_encoded'].fillna(global_mean_savings, inplace=True)
df_test['destination_airport_encoded'].fillna(global_mean_savings, inplace=True)


In [6]:
import pandas as pd
import numpy as np

# Assuming df_train and df_test are already defined

# Step 1: Calculate the global mean of the target variable
global_mean_savings = df_enriched['mean_savings'].mean()

# Step 2: Choose a smoothing parameter
alpha = 10

# Preparation for Steps 3 & 4: Calculate the number of observations and mean for each category
agg_departure = df_enriched.groupby('departure_airport_code')['mean_savings'].agg(['mean', 'size'])
agg_destination = df_enriched.groupby('destination_airport_code')['mean_savings'].agg(['mean', 'size'])

# Step 4: Apply smoothing formula
departure_airport_encoded = (agg_departure['mean'] * agg_departure['size'] + global_mean_savings * alpha) / (agg_departure['size'] + alpha)
destination_airport_encoded = (agg_destination['mean'] * agg_destination['size'] + global_mean_savings * alpha) / (agg_destination['size'] + alpha)

# Step 5: Map the smoothed encodings onto both training and test datasets
df_enriched['departure_airport_encoded'] = df_enriched['departure_airport_code'].map(departure_airport_encoded)
df_enriched['destination_airport_encoded'] = df_enriched['destination_airport_code'].map(destination_airport_encoded)

df_test['departure_airport_encoded'] = df_test['departure_airport_code'].map(departure_airport_encoded)
df_test['destination_airport_encoded'] = df_test['destination_airport_code'].map(destination_airport_encoded)

# Step 6: Handle potential missing values for unseen categories in the test set
df_test['departure_airport_encoded'].fillna(global_mean_savings, inplace=True)
df_test['destination_airport_encoded'].fillna(global_mean_savings, inplace=True)


In [7]:
# Step 1: Calculate the mean of 'mean_savings' for each 'Detected_Country' in the training set
country_mean_savings = df_enriched.groupby('Detected_Country')['mean_savings'].mean()

# Step 2: Map these mean values onto both training and test datasets
df_enriched['Detected_Country_encoded'] = df_enriched['Detected_Country'].map(country_mean_savings)
df_test['Detected_Country_encoded'] = df_test['Detected_Country'].map(country_mean_savings)

# Optional: Handle potential missing values for unseen categories in the test set
# This step is only necessary if there are countries in the test set not seen in the training set
df_test['Detected_Country_encoded'].fillna(global_mean_savings, inplace=True)


In [16]:
from sklearn.model_selection import train_test_split
import xgboost as xgb

X_train = df_enriched[['departure_airport_encoded','destination_airport_encoded', 'Detected_Country_encoded']]
y_train = df_enriched['mean_savings']




# Specify your model parameters here
params_grid = {'colsample_bytree': 0.9, 'eta': 0.299, 'max_depth': 7, 'subsample': 0.8}
params_optuna = {'max_depth': 7, 'eta': 0.319923163390001, 'subsample': 0.8, 'colsample_bytree': 0.9}
params_optuna200 = {'max_depth': 8, 'eta': 0.34991201767803204, 'subsample': 0.9, 'colsample_bytree': 0.9}


params_self = {'max_depth': 3, 'eta': 0.01, 'subsample': 0.9, 'colsample_bytree': 0.9}




# Create XGBoost model
model_v8_combined  = xgb.XGBRegressor(**params_self)

# Fit the model on your training data
model_v8_combined.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9, device=None, early_stopping_rounds=None,
             enable_categorical=False, eta=0.01, eval_metric=None,
             feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [17]:
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
# Assuming new_test_df is your new test data


# Update X_test and y_test accordingly
X_test = df_test[['departure_airport_encoded','destination_airport_encoded', 'Detected_Country_encoded']]
y_test = df_test['mean_savings']

"""""
# Align the new test set features with the training features
# Add missing columns in the new test set
missing_cols = set(feature_names) - set(X_test.columns)
for c in missing_cols:
    X_test[c] = 0

# Remove any columns in the new test set that are not present in the training set
X_test = X_test[feature_names]
"""


# Now, you can predict with your model
y_pred = model_v8_combined.predict(X_test)


# Calculate and print the Mean Squared Error
mea = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mea}")


r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")

Mean Absolute Error: 2.7386126710478043
R-squared: 0.027655535164319622


# Smoothed Mean

In [3]:
import pandas as pd
df_1 = pd.read_csv('../data/4.processed_data/Query4_results_test_processed.csv')
df_2 = pd.read_csv('../data/4.processed_data/Query1503_results_conv_processed.csv')
df_3 = pd.read_csv('../data/4.processed_data/Query1603_results_test_processed.csv')
df_enriched= pd.concat([df_1,df_2,df_3])
df_test = pd.read_csv('../data/4.processed_data/Query1803_results_processed.csv')

In [4]:
# Step 1: Calculate the global mean
global_mean_train = df_enriched['rel_diff_to_min_price_FlightID'].mean()

# Step 2: Calculate group sizes and means
group_stats = df_enriched.groupby(['Journey_ID', 'Detected_Country'])['rel_diff_to_min_price_FlightID'].agg(['mean', 'size'])

# Choose a smoothing parameter (alpha)
alpha = 10

# Step 3: Apply the smoothing formula
# The smoothed mean is a weighted average of the global mean and the group mean, with weights depending on the group size and alpha
smoothed_means = (group_stats['mean'] * group_stats['size'] + global_mean_train * alpha) / (group_stats['size'] + alpha)

# Convert smoothed_means to a DataFrame and reset the index so it can be merged
smoothed_means_df = smoothed_means.reset_index(name='smoothed_mean')

# Merge the smoothed means back into df_reduced
df_enriched = df_enriched.merge(smoothed_means_df, on=['Journey_ID', 'Detected_Country'], how='left')


In [5]:
# Step 1: Calculate the global mean
global_mean_test = df_test['rel_diff_to_min_price_FlightID'].mean()

# Step 2: Calculate group sizes and means
group_stats = df_test.groupby(['Journey_ID', 'Detected_Country'])['rel_diff_to_min_price_FlightID'].agg(['mean', 'size'])

# Choose a smoothing parameter (alpha)
alpha = 20

# Step 3: Apply the smoothing formula
# The smoothed mean is a weighted average of the global mean and the group mean, with weights depending on the group size and alpha
smoothed_means = (group_stats['mean'] * group_stats['size'] + global_mean_test * alpha) / (group_stats['size'] + alpha)

# Convert smoothed_means to a DataFrame and reset the index so it can be merged
smoothed_means_df = smoothed_means.reset_index(name='smoothed_mean')

# Merge the smoothed means back into df_reduced
df_test = df_test.merge(smoothed_means_df, on=['Journey_ID', 'Detected_Country'], how='left')

In [6]:
from sklearn.model_selection import train_test_split
import xgboost as xgb

X_train = pd.get_dummies(df_enriched[['Journey_route','days_until_departure', 'Detected_Country']])
y_train = df_enriched['smoothed_mean']
feature_names = X_train.columns.tolist()



# Specify your model parameters here
params_grid = {'colsample_bytree': 0.9, 'eta': 0.299, 'max_depth': 7, 'subsample': 0.8}
params_optuna = {'max_depth': 7, 'eta': 0.319923163390001, 'subsample': 0.8, 'colsample_bytree': 0.9}
params_optuna200 = {'max_depth': 8, 'eta': 0.34991201767803204, 'subsample': 0.9, 'colsample_bytree': 0.9}


params_self = {'max_depth': 3, 'eta': 0.01, 'subsample': 0.9, 'colsample_bytree': 0.9}




# Create XGBoost model
model_v8_combined  = xgb.XGBRegressor(**params_optuna200)

# Fit the model on your training data
model_v8_combined.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9, device=None, early_stopping_rounds=None,
             enable_categorical=False, eta=0.34991201767803204,
             eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [7]:
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
# Assuming new_test_df is your new test data


# Update X_test and y_test accordingly
X_test = pd.get_dummies(df_test[['Journey_route', 'days_until_departure', 'Detected_Country']])
y_test = df_test['smoothed_mean']


# Align the new test set features with the training features
# Add missing columns in the new test set
missing_cols = set(feature_names) - set(X_test.columns)
for c in missing_cols:
    X_test[c] = 0

# Remove any columns in the new test set that are not present in the training set
X_test = X_test[feature_names]

# Now, you can predict with your model
y_pred = model_v8_combined.predict(X_test)



# Calculate and print the Mean Squared Error
mea = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mea}")


r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")

Mean Absolute Error: 0.8352616192708411
R-squared: -1.0766354893630354
